In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

## 配置GPU

In [2]:
device  = torch.device('cuda' if torch.cuda.is_available else 'cpu')

## 超参数

In [3]:
num_epochs = 80
learning_rate = 0.001

## 图像预处理模组

In [4]:
transform = transforms.Compose([
    transforms.Pad(4),#填充边框
    transforms.RandomHorizontalFlip(),#水平翻转
    transforms.RandomCrop(32),#切割中心点的位置随机选取
    transforms.ToTensor()#PIL.Image转为Torch张量
])

## CIFAR-10 数据集

In [6]:
train_dataset = torchvision.datasets.CIFAR10(root='../../data/CIFAR-10',
                                            train=True,
                                            transform=transform,
                                            download=False)
test_dataset = torchvision.datasets.CIFAR10(root='../../data/CIFAR-10',
                                           train=False,
                                           transform=transforms.ToTensor(),
                                           download=False)

## 数据生成器DataLoader

In [7]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=100,
                                          shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=100,
                                         shuffle=False)

## 构造 3*3 卷积

In [8]:
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3,
                    stride=stride, padding=1, bias=False)

## 构造残差模组

In [9]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)#下采样，残差
        out += residual
        out = self.relu(out)
        return out

## 构造ResNet——残差网络

In [21]:
# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)#*加上形参名的方式来表示这个函数 的实参个数不定，在函数内部都被存放在以形参名为标识符的tuple中。
    
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [22]:
model = ResNet(ResidualBlock, [2, 2, 2, 2]).to(device)

## 构造 损失 和 优化器

In [23]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## 更新学习率

In [24]:
def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr#.param_group中保存了参数组及其对应的学习率,动量等等.所以我们可以通过更改param_group[‘lr’]的值来更改对应参数组的学习率

## 训练模型

In [25]:
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        # 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 反向传播 和 优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # 学习率衰减
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

Epoch [1/80], Step [100/500] Loss: 1.5748
Epoch [1/80], Step [200/500] Loss: 1.4621
Epoch [1/80], Step [300/500] Loss: 1.3846
Epoch [1/80], Step [400/500] Loss: 1.2582
Epoch [1/80], Step [500/500] Loss: 1.1926
Epoch [2/80], Step [100/500] Loss: 0.9526
Epoch [2/80], Step [200/500] Loss: 1.1695
Epoch [2/80], Step [300/500] Loss: 1.0592
Epoch [2/80], Step [400/500] Loss: 1.0909
Epoch [2/80], Step [500/500] Loss: 0.9920
Epoch [3/80], Step [100/500] Loss: 0.9477
Epoch [3/80], Step [200/500] Loss: 1.0780
Epoch [3/80], Step [300/500] Loss: 1.0453
Epoch [3/80], Step [400/500] Loss: 0.9746
Epoch [3/80], Step [500/500] Loss: 0.9332
Epoch [4/80], Step [100/500] Loss: 1.0036
Epoch [4/80], Step [200/500] Loss: 1.0481
Epoch [4/80], Step [300/500] Loss: 0.6685
Epoch [4/80], Step [400/500] Loss: 0.6394
Epoch [4/80], Step [500/500] Loss: 0.7084
Epoch [5/80], Step [100/500] Loss: 0.7055
Epoch [5/80], Step [200/500] Loss: 0.6627
Epoch [5/80], Step [300/500] Loss: 0.4535
Epoch [5/80], Step [400/500] Loss:

Epoch [39/80], Step [200/500] Loss: 0.2022
Epoch [39/80], Step [300/500] Loss: 0.3298
Epoch [39/80], Step [400/500] Loss: 0.3501
Epoch [39/80], Step [500/500] Loss: 0.2492
Epoch [40/80], Step [100/500] Loss: 0.1636
Epoch [40/80], Step [200/500] Loss: 0.1156
Epoch [40/80], Step [300/500] Loss: 0.2458
Epoch [40/80], Step [400/500] Loss: 0.1707
Epoch [40/80], Step [500/500] Loss: 0.1913
Epoch [41/80], Step [100/500] Loss: 0.2228
Epoch [41/80], Step [200/500] Loss: 0.1188
Epoch [41/80], Step [300/500] Loss: 0.1215
Epoch [41/80], Step [400/500] Loss: 0.2155
Epoch [41/80], Step [500/500] Loss: 0.1110
Epoch [42/80], Step [100/500] Loss: 0.1514
Epoch [42/80], Step [200/500] Loss: 0.1646
Epoch [42/80], Step [300/500] Loss: 0.2198
Epoch [42/80], Step [400/500] Loss: 0.2520
Epoch [42/80], Step [500/500] Loss: 0.1769
Epoch [43/80], Step [100/500] Loss: 0.1584
Epoch [43/80], Step [200/500] Loss: 0.2582
Epoch [43/80], Step [300/500] Loss: 0.1059
Epoch [43/80], Step [400/500] Loss: 0.2283
Epoch [43/8

Epoch [77/80], Step [200/500] Loss: 0.2776
Epoch [77/80], Step [300/500] Loss: 0.1860
Epoch [77/80], Step [400/500] Loss: 0.2562
Epoch [77/80], Step [500/500] Loss: 0.2118
Epoch [78/80], Step [100/500] Loss: 0.1411
Epoch [78/80], Step [200/500] Loss: 0.1329
Epoch [78/80], Step [300/500] Loss: 0.1104
Epoch [78/80], Step [400/500] Loss: 0.1810
Epoch [78/80], Step [500/500] Loss: 0.3269
Epoch [79/80], Step [100/500] Loss: 0.1829
Epoch [79/80], Step [200/500] Loss: 0.0803
Epoch [79/80], Step [300/500] Loss: 0.1459
Epoch [79/80], Step [400/500] Loss: 0.0816
Epoch [79/80], Step [500/500] Loss: 0.1490
Epoch [80/80], Step [100/500] Loss: 0.1704
Epoch [80/80], Step [200/500] Loss: 0.1283
Epoch [80/80], Step [300/500] Loss: 0.0978
Epoch [80/80], Step [400/500] Loss: 0.1558
Epoch [80/80], Step [500/500] Loss: 0.1820


## 保存模型检查点

In [26]:
torch.save(model.state_dict(), 'resnet.ckpt')